# Logistic Regression on CVD

In [1]:
#import dependencies
import pandas as pd

In [2]:
#read in data set
df = pd.read_csv("Resources/cvd_period1_data.csv")
df.head(20)

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,PREVAP,PREVHYP,DEATH,CVD
0,2448,1,195.0,39,106.0,70.0,0,0.0,26.97,0,0.0,80.0,77.0,0,0,0,1
1,6238,2,250.0,46,121.0,81.0,0,0.0,28.73,0,0.0,95.0,76.0,0,0,0,0
2,9428,1,245.0,48,127.5,80.0,1,20.0,25.34,0,0.0,75.0,70.0,0,0,0,0
3,10552,2,225.0,61,150.0,95.0,1,30.0,28.58,0,0.0,65.0,103.0,0,1,1,1
4,11252,2,285.0,46,130.0,84.0,1,23.0,23.10,0,0.0,85.0,85.0,0,0,0,0
5,11263,2,228.0,43,180.0,110.0,0,0.0,30.30,0,0.0,77.0,99.0,0,1,0,1
6,12629,2,205.0,63,138.0,71.0,0,0.0,33.11,0,0.0,60.0,85.0,0,0,0,0
7,12806,2,313.0,45,100.0,71.0,1,20.0,21.68,0,0.0,79.0,78.0,0,0,0,0
8,14367,1,260.0,52,141.5,89.0,0,0.0,26.36,0,0.0,76.0,79.0,0,1,0,0
9,16365,1,225.0,43,162.0,107.0,1,30.0,23.61,0,0.0,93.0,88.0,0,1,0,0


In [3]:
cvd_df = df.set_index('RANDID')
cvd_df.head()

,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,PREVAP,PREVHYP,DEATH,CVD
RANDID,,,,,,,,,,,,,,,,
2448,1,195.0,39,106.0,70.0,0,0.0,26.97,0,0.0,80.0,77.0,0,0,0,1
6238,2,250.0,46,121.0,81.0,0,0.0,28.73,0,0.0,95.0,76.0,0,0,0,0
9428,1,245.0,48,127.5,80.0,1,20.0,25.34,0,0.0,75.0,70.0,0,0,0,0
10552,2,225.0,61,150.0,95.0,1,30.0,28.58,0,0.0,65.0,103.0,0,1,1,1
11252,2,285.0,46,130.0,84.0,1,23.0,23.10,0,0.0,85.0,85.0,0,0,0,0


In [7]:
# Number of hospitalizations for myocardial infarctions and number of non-events
cvd_count=cvd_df.loc[(cvd_df['CVD']==1)]
no_cvd_count = cvd_df.loc[(cvd_df['CVD']==0)]

print(f"Subjects hospitalized with cvd: {len(cvd_count)}")
print(f"Subjects with no cvd events: {len(no_cvd_count)}")

Subjects hospitalized with cvd: 897
Subjects with no cvd events: 2833


In [8]:
cvd_df.columns

Index(['SEX', 'TOTCHOL', 'AGE', 'SYSBP', 'DIABP', 'CURSMOKE', 'CIGPDAY', 'BMI',
       'DIABETES', 'BPMEDS', 'HEARTRTE', 'GLUCOSE', 'PREVAP', 'PREVHYP',
       'DEATH', 'CVD'],
      dtype='object')

In [9]:
#Remove columns not involved in analysis
#removed 'Time', 'Period', 'Death', 'MI_FCHD', 'TIMEMI', 'TIMEMIFC'
cvd_reg_df = cvd_df[['SEX', 'TOTCHOL', 'AGE', 'SYSBP', 'DIABP', 'CURSMOKE', 'CIGPDAY', 'BMI',
                     'DIABETES', 'BPMEDS', 'HEARTRTE', 'GLUCOSE', 'PREVAP', 'PREVHYP', 'CVD']]
cvd_reg_df.head()

,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,PREVAP,PREVHYP,CVD
RANDID,,,,,,,,,,,,,,,
2448,1,195.0,39,106.0,70.0,0,0.0,26.97,0,0.0,80.0,77.0,0,0,1
6238,2,250.0,46,121.0,81.0,0,0.0,28.73,0,0.0,95.0,76.0,0,0,0
9428,1,245.0,48,127.5,80.0,1,20.0,25.34,0,0.0,75.0,70.0,0,0,0
10552,2,225.0,61,150.0,95.0,1,30.0,28.58,0,0.0,65.0,103.0,0,1,1
11252,2,285.0,46,130.0,84.0,1,23.0,23.10,0,0.0,85.0,85.0,0,0,0


In [10]:
# Assign X (data) and y (target)
X = cvd_reg_df.drop('CVD', axis=1)
y= cvd_reg_df['CVD']
print(X.shape, y.shape)

(3730, 14) (3730,)


In [11]:
# Split the data using train_test_split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
# Create a logistic regression model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [13]:
# Fit the model to the data
classifier.fit(X_train, y_train)

C:\Users\jtaum\Anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [14]:
#Validate the model using test data
# Print the r2 score for the test data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.7547372184483375
Testing Data Score: 0.7845659163987139


In [15]:
#Make predictions (0 = no event occurred, 1 = event occurred)
predictions = classifier.predict(X_test)
cvd_predictions = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
cvd_predictions

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
928,0,1
929,0,1
930,0,1
931,0,0


In [17]:
p0a0 = cvd_predictions.loc[(cvd_predictions['Prediction']==0) & (cvd_predictions['Actual']==0)]
p1a1 = cvd_predictions.loc[(cvd_predictions['Prediction']==1) & (cvd_predictions['Actual']==1)]
p0a1 = cvd_predictions.loc[(cvd_predictions['Prediction']==0) & (cvd_predictions['Actual']==1)]
p1a0 = cvd_predictions.loc[(cvd_predictions['Prediction']==1) & (cvd_predictions['Actual']==0)]

In [19]:
print(f"Event prediction = 0 and actual = 0:  {len(p0a0)}")
print(f"Event prediction = 1 and actual = 1:  {len(p1a1)}")
print(f"Event prediction = 0 and actual = 1:  {len(p0a1)}")
print(f"Event prediction = 1 and actual = 0:  {len(p1a0)}")

Event prediction = 0 and actual = 0:  697
Event prediction = 1 and actual = 1:  35
Event prediction = 0 and actual = 1:  172
Event prediction = 1 and actual = 0:  29


## CVD period 3 logistic regression

In [20]:
period3_df = pd.read_csv("Resources/cvd_period3_data.csv")
period3_df

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,PREVAP,PREVHYP,HDLC,LDLC,DEATH,CVD
0,6238,2,237.0,58,108.0,66.0,0,0.0,28.50,0,0.0,80.0,71.0,0,0,54.0,141.0,0,0
1,11263,2,220.0,55,180.0,106.0,0,0.0,31.17,1,1.0,86.0,81.0,0,1,46.0,135.0,0,1
2,12806,2,320.0,57,110.0,46.0,1,30.0,22.02,0,0.0,75.0,87.0,0,0,34.0,286.0,0,0
3,14367,1,280.0,64,168.0,100.0,0,0.0,25.72,0,0.0,92.0,82.0,0,1,44.0,236.0,0,0
4,16365,1,211.0,55,173.0,123.0,0,0.0,29.11,0,1.0,75.0,85.0,0,1,48.0,163.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,9978986,2,243.0,68,110.0,66.0,0,0.0,24.30,0,0.0,80.0,108.0,0,1,65.0,178.0,0,0
2022,9982118,1,219.0,70,163.5,75.0,0,0.0,25.26,0,0.0,65.0,103.0,0,1,50.0,169.0,1,1
2023,9990894,2,228.0,60,205.0,100.0,1,20.0,22.37,0,0.0,80.0,91.0,0,1,47.0,181.0,1,0
2024,9993179,2,251.0,56,145.0,92.0,1,35.0,21.97,0,1.0,95.0,90.0,0,1,70.0,181.0,1,0


In [21]:
cvd3_df = period3_df.set_index('RANDID')
cvd3_df.head()

,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,PREVAP,PREVHYP,HDLC,LDLC,DEATH,CVD
RANDID,,,,,,,,,,,,,,,,,,
6238,2,237.0,58,108.0,66.0,0,0.0,28.50,0,0.0,80.0,71.0,0,0,54.0,141.0,0,0
11263,2,220.0,55,180.0,106.0,0,0.0,31.17,1,1.0,86.0,81.0,0,1,46.0,135.0,0,1
12806,2,320.0,57,110.0,46.0,1,30.0,22.02,0,0.0,75.0,87.0,0,0,34.0,286.0,0,0
14367,1,280.0,64,168.0,100.0,0,0.0,25.72,0,0.0,92.0,82.0,0,1,44.0,236.0,0,0
16365,1,211.0,55,173.0,123.0,0,0.0,29.11,0,1.0,75.0,85.0,0,1,48.0,163.0,0,0


In [22]:
# Number of hospitalizations for cvd and number of non-events
cvd_count=cvd3_df.loc[(cvd3_df['CVD']==1)]
no_cvd_count = cvd3_df.loc[(cvd3_df['CVD']==0)]

print(f"Subjects hospitalized with cvd: {len(cvd_count)}")
print(f"Subjects with no cvd events: {len(no_cvd_count)}")

Subjects hospitalized with cvd: 332
Subjects with no cvd events: 1694


In [23]:
cvd3_df.columns

Index(['SEX', 'TOTCHOL', 'AGE', 'SYSBP', 'DIABP', 'CURSMOKE', 'CIGPDAY', 'BMI',
       'DIABETES', 'BPMEDS', 'HEARTRTE', 'GLUCOSE', 'PREVAP', 'PREVHYP',
       'HDLC', 'LDLC', 'DEATH', 'CVD'],
      dtype='object')

In [25]:
#Remove columns not involved in analysis
#removed 'Time', 'Period', 'Death', 'MI_FCHD', 'TIMEMI', 'TIMEMIFC'
cvd3_reg_df = cvd3_df[['SEX', 'TOTCHOL', 'AGE', 'SYSBP', 'DIABP', 'CURSMOKE', 'CIGPDAY', 'BMI',
       'DIABETES', 'BPMEDS', 'HEARTRTE', 'GLUCOSE', 'PREVAP', 'PREVHYP',
       'HDLC', 'LDLC', 'CVD']]
cvd3_reg_df.head()

,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,PREVAP,PREVHYP,HDLC,LDLC,CVD
RANDID,,,,,,,,,,,,,,,,,
6238,2,237.0,58,108.0,66.0,0,0.0,28.50,0,0.0,80.0,71.0,0,0,54.0,141.0,0
11263,2,220.0,55,180.0,106.0,0,0.0,31.17,1,1.0,86.0,81.0,0,1,46.0,135.0,1
12806,2,320.0,57,110.0,46.0,1,30.0,22.02,0,0.0,75.0,87.0,0,0,34.0,286.0,0
14367,1,280.0,64,168.0,100.0,0,0.0,25.72,0,0.0,92.0,82.0,0,1,44.0,236.0,0
16365,1,211.0,55,173.0,123.0,0,0.0,29.11,0,1.0,75.0,85.0,0,1,48.0,163.0,0


In [26]:
# Assign X (data) and y (target)
X = cvd3_reg_df.drop('CVD', axis=1)
y= cvd3_reg_df['CVD']
print(X.shape, y.shape)

(2026, 16) (2026,)


In [27]:
# Split the data using train_test_split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=44)

In [28]:
# Create a logistic regression model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [29]:
# Fit the model to the data
classifier.fit(X_train, y_train)

C:\Users\jtaum\Anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [30]:
#Validate the model using test data
# Print the r2 score for the test data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.8393680052666228
Testing Data Score: 0.8303747534516766


In [31]:
#Make predictions (0 = no event occurred, 1 = event occurred)
predictions = classifier.predict(X_test)
cvd_predictions = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
cvd_predictions

,Prediction,Actual
0,0,0
1,0,0
2,0,1
3,0,1
4,0,0
...,...,...
502,0,0
503,0,0
504,0,0
505,0,0


In [32]:
p0a0 = cvd_predictions.loc[(cvd_predictions['Prediction']==0) & (cvd_predictions['Actual']==0)]
p1a1 = cvd_predictions.loc[(cvd_predictions['Prediction']==1) & (cvd_predictions['Actual']==1)]
p0a1 = cvd_predictions.loc[(cvd_predictions['Prediction']==0) & (cvd_predictions['Actual']==1)]
p1a0 = cvd_predictions.loc[(cvd_predictions['Prediction']==1) & (cvd_predictions['Actual']==0)]

print(f"Event prediction = 0 and actual = 0:  {len(p0a0)}")
print(f"Event prediction = 1 and actual = 1:  {len(p1a1)}")
print(f"Event prediction = 0 and actual = 1:  {len(p0a1)}")
print(f"Event prediction = 1 and actual = 0:  {len(p1a0)}")

Event prediction = 0 and actual = 0:  415
Event prediction = 1 and actual = 1:  6
Event prediction = 0 and actual = 1:  83
Event prediction = 1 and actual = 0:  3
